In [10]:
# import os
# import pandas as pd

# temp_list_df = []
# dir_path = os.path.join(os.getcwd(),'data_scrap')

# for path in os.listdir(dir_path):
#     if os.path.isfile(os.path.join(dir_path, path)):
#         temp_list_df.append(pd.read_csv(os.path.join(dir_path,path)))

# df = pd.concat(temp_list_df)
# df.to_csv('data_scrap_combined.csv',index=False)

In [11]:
import pandas as pd
import os
import re
import string

In [12]:
df = pd.read_csv('data_scrap_combined.csv')
df.head()

,base_type,type_,text,price
0,handphone-tablet,aksesoris-handphone,Ugreen Data Cable Charger Fast Charging 3A Mic...,Rp59.900
1,handphone-tablet,aksesoris-handphone,Power Adapter iphone USB-C 20w Fast charging a...,Rp115.000
2,handphone-tablet,aksesoris-handphone,Charger Samsung Super Fast Charging Original A...,Rp125.000
3,handphone-tablet,aksesoris-handphone,TONGSIS BLUETOOTH WITH TRIPOD AND ROTATION HOL...,Rp154.875
4,handphone-tablet,aksesoris-handphone,GUB P30 Original Holder Hp Dudukan Ponsel Alum...,Rp108.000


In [13]:
def text_cleaning(text:str)->str:
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    # text = re.sub('\w*\d\w*', '', text)
    return text

def price_to_int(text:str):
    text = text.replace('Rp','').replace('.','')
    return int(text)

In [14]:
df['text_cleaned'] = df['text'].apply(lambda x:text_cleaning(x))
df['price_int'] = df['price'].apply(lambda x:price_to_int(x))

In [15]:
df_cleaned = df[['text_cleaned','price_int','type_']]

In [16]:
df_cleaned['type_']

0         aksesoris-handphone
1         aksesoris-handphone
2         aksesoris-handphone
3         aksesoris-handphone
4         aksesoris-handphone
                 ...         
126445       wearable-devices
126446       wearable-devices
126447       wearable-devices
126448       wearable-devices
126449       wearable-devices
Name: type_, Length: 126450, dtype: object

In [17]:
import os
import pandas as pd
import numpy as np
from scipy.stats import randint
import seaborn as sns # used for plot interactive graph. 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
#import warnings
#warnings.filterwarnings("ignore", category=FutureWarning)

In [18]:
unique_col = df_cleaned['type_'].unique()
def to_category_id(x, unique_col):
    count = 0
    for col in unique_col:
        if str(x) ==  str(col):
            return count
        count+=1
df_cleaned['category_id'] = df_cleaned['type_'].apply(lambda x : to_category_id(x,unique_col))

C:\Users\Andrey\AppData\Local\Temp\ipykernel_16268\1051128527.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['category_id'] = df_cleaned['type_'].apply(lambda x : to_category_id(x,unique_col))


# Using Machine Learning Solution

In [ ]:
tfidf = TfidfVectorizer(sublinear_tf=True, 
                        min_df=5,
                        max_features=1000)

# We transform each complaint into a vector
features = tfidf.fit_transform(df_cleaned['text_cleaned']).toarray()

labels = df_cleaned['category_id']

In [23]:
# longer max_features longer time to compute
models = [
    RandomForestClassifier(n_estimators=100, max_depth=5, random_state=0),
    MultinomialNB(),
    LogisticRegression(random_state=0),
]

# 2 Cross-validation
CV = 2
cv_df = pd.DataFrame(index=range(CV * len(models)))

entries = []
for model in models:
  model_name = model.__class__.__name__
  accuracies = cross_val_score(model, features, labels, scoring='accuracy', cv=CV)
  for fold_idx, accuracy in enumerate(accuracies):
    entries.append((model_name, fold_idx, accuracy))
    
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])

c:\Users\Andrey\miniconda3\envs\rakamin2\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\Andrey\miniconda3\envs\rakamin2\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.ht

In [122]:
cv_df

,model_name,fold_idx,accuracy
0,RandomForestClassifier,0,0.561645
1,RandomForestClassifier,1,0.532084
2,MultinomialNB,0,0.833009
3,MultinomialNB,1,0.822681
4,LogisticRegression,0,0.861321
5,LogisticRegression,1,0.852985


In [135]:
df_cleaned.head()

,text_cleaned,price_int,type_,category_id
0,ugreen data cable charger fast charging 3a mic...,59900,aksesoris-handphone,0
1,power adapter iphone usbc 20w fast charging ad...,115000,aksesoris-handphone,0
2,charger samsung super fast charging original a...,125000,aksesoris-handphone,0
3,tongsis bluetooth with tripod and rotation hol...,154875,aksesoris-handphone,0
4,gub p30 original holder hp dudukan ponsel alum...,108000,aksesoris-handphone,0


In [136]:
X = df_cleaned[['text_cleaned','price_int']] # Collection of documents
y = df_cleaned['type_'] # Target or the labels we want to predict (i.e., the 13 different complaints of products)
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.25,
                                                    random_state = 0)


In [148]:
tfidf = TfidfVectorizer(sublinear_tf=True, 
                        min_df=5,
                        max_features=1000)

fitted_vectorizer = tfidf.fit(X_train['text_cleaned'])
tfidf_vectorizer_vectors = fitted_vectorizer.transform(X_train['text_cleaned']).toarray()

In [145]:
# from sklearn.preprocessing import StandardScaler

# s_scaler = StandardScaler()
# fitted_scaler = s_scaler.fit(X_train[['price_int']])
# s_scaler_vector = fitted_scaler.transform(X_train[['price_int']])

In [165]:
# stacked_vector = np.column_stack((tfidf_vectorizer_vectors, s_scaler_vector))

In [166]:
model = LogisticRegression().fit(tfidf_vectorizer_vectors, y_train)

c:\Users\Andrey\miniconda3\envs\rakamin2\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [167]:
from sklearn.metrics import classification_report

tfidf_vectorizer_vectors_test = fitted_vectorizer.transform(X_test['text_cleaned']).toarray()
# s_scaler_vector_test = fitted_scaler.transform(X_test[['price_int']])
# stacked_vector_test = np.column_stack((tfidf_vectorizer_vectors_test, s_scaler_vector_test))
y_pred = model.predict(tfidf_vectorizer_vectors_test)
print(classification_report(y_test, y_pred, target_names=model.classes_))

                           precision    recall  f1-score   support

   aksesoris-game-console       0.83      0.79      0.81      1417
      aksesoris-handphone       0.76      0.93      0.84      2327
aksesoris-komputer-laptop       0.81      0.79      0.80      1484
  aksesoris-mobile-gaming       0.84      0.81      0.83      1567
      aksesoris-pc-gaming       0.89      0.84      0.86      1516
         aksesoris-tablet       0.82      0.57      0.68      1261
   alat-pendingin-ruangan       0.95      0.94      0.95      1397
                    audio       0.78      0.87      0.82      1442
                  cd-game       0.85      0.90      0.87      1526
          desktop-mini-pc       0.89      0.86      0.87      1446
         elektronik-dapur       0.92      0.92      0.92      1339
  elektronik-rumah-tangga       0.95      0.91      0.93      1496
             game-console       0.79      0.76      0.77      1387
                handphone       0.86      0.86      0.86     

In [ ]:
# todo use numeric feature also

# Extract Important Keywords

In [28]:
df_train = pd.DataFrame()
df_train["text_cleaned"] = X_train.copy()
df_train["type_"] = y_train.copy()
df_train.head()

,text_cleaned,type_
18532,kabel audio aux iphone lighting usb to jack 35...,audio
110584,acer aspire 3 slim a314 athlon 3020 4gb 256ssd...,laptop
2021,100 original xiaomi mi selfie stick tripod blu...,aksesoris-handphone
52813,pulse 3d wireless headset for playstation 5 ps...,aksesoris-game-console
54089,mc memory card playstation ps2 8mb 16mb 32mb,aksesoris-game-console


In [29]:
df_cleaned.head()

,text_cleaned,price_int,type_,category_id
0,ugreen data cable charger fast charging 3a mic...,59900,aksesoris-handphone,0
1,power adapter iphone usbc 20w fast charging ad...,115000,aksesoris-handphone,0
2,charger samsung super fast charging original a...,125000,aksesoris-handphone,0
3,tongsis bluetooth with tripod and rotation hol...,154875,aksesoris-handphone,0
4,gub p30 original holder hp dudukan ponsel alum...,108000,aksesoris-handphone,0


In [30]:
df_group = df_cleaned.groupby('type_')['text_cleaned'].apply(list).reset_index(name='text_cleaned_collection')

In [31]:
x = " ".join(list(set(df_group['text_cleaned_collection'].iloc[0]))).split()
pd.DataFrame(x)[0].value_counts()

stik          1782
switch        1439
nintendo      1337
stick         1150
ps4           1009
              ... 
600rb            1
600000           1
ps3ps2ps4        1
switchmlal       1
1pasang          1
Name: 0, Length: 3879, dtype: int64

In [32]:
df_group["text_cleaned_collection"] = df_group["text_cleaned_collection"].apply(lambda x: list(set(x)))
df_group["text_cleaned_collection"] = df_group["text_cleaned_collection"].apply(lambda x: " ".join(x))
df_group.head()

,type_,text_cleaned_collection
0,aksesoris-game-console,stik stick joystick gamepad xbox one pc laptop...
1,aksesoris-handphone,tempered glass 9d oppo a7a9a5sa11k f9f11anti g...
2,aksesoris-komputer-laptop,vention hdmi extender repeater amplifier boost...
3,aksesoris-mobile-gaming,stik stick joystick gamepad xbox one pc laptop...
4,aksesoris-pc-gaming,razer deathadder v2 x hyperspeed ergonomic wi...


In [33]:
len(df_group['text_cleaned_collection'].iloc[0].split())

48307

In [34]:
tfidf = TfidfVectorizer(sublinear_tf=True, lowercase=True)
fitted_vectorizer = tfidf.fit(df_group["text_cleaned_collection"])
tfidf_vectorizer_vectors = fitted_vectorizer.transform(df_group["text_cleaned_collection"])

In [35]:
def extract_keywords_tfidf(doc, tfidf : TfidfVectorizer, topk=10):
    vec = tfidf.transform([doc])
    vocab = tfidf.get_feature_names_out()
    sorted_tfidf = vec.toarray().argsort()
    return [vocab[idx] for idx in reversed(sorted_tfidf[0, -topk:])]

In [36]:
MAX_KEYWORDS = 20
df_group["important_keywords"] = df_group["text_cleaned_collection"].apply(lambda text:" ".join(extract_keywords_tfidf(text, tfidf, topk=MAX_KEYWORDS)))

In [37]:
df_group.head()

,type_,text_cleaned_collection,important_keywords
0,aksesoris-game-console,stik stick joystick gamepad xbox one pc laptop...,joycon psp ds4 ps3 ps2 joy con dualsense plays...
1,aksesoris-handphone,tempered glass 9d oppo a7a9a5sa11k f9f11anti g...,24a tongsis qc30 privacy ringke cafule qc hydr...
2,aksesoris-komputer-laptop,vention hdmi extender repeater amplifier boost...,19v 342a writing tulis webcam 237a a2681 325a ...
3,aksesoris-mobile-gaming,stik stick joystick gamepad xbox one pc laptop...,sinyal gamepad ipega bosetech repeater gladius...
4,aksesoris-pc-gaming,razer deathadder v2 x hyperspeed ergonomic wi...,xierra tkl meca mousepad chair vortexseries ar...


## Calculate Similarity (Bebas pake apa aja)

In [38]:
from thefuzz import fuzz
from thefuzz import process

In [3]:
choices = ["Atlanta Falcons", "New York Jets", "New York Giants", "Dallas Cowboys"]
process.extract("new york jets!", choices, limit=2)

[('New York Jets', 100), ('New York Giants', 79)]

In [83]:
test_title = "Microsoft Office 2016 Pro Plus Original Lifetime - For Win PC/Laptop - DVD"
test_title = test_title.lower()
keywords = df_group["important_keywords"]
test_title

'microsoft office 2016 pro plus original lifetime - for win pc/laptop - dvd'

In [84]:
process.extract(test_title, keywords, limit=1) # Text, Score, Index

[('e27 downlight hannochs bohlam multipack 6500k sorot 8w cob mycare 9w 7w outbow luby inlite bulb 11w 4000k tenaga cdl',
  35,
  15)]

In [42]:
df_test = pd.DataFrame()
df_test["text_cleaned"] = X_test.copy()
df_test["type_"] = y_test.copy()
df_test.head()

,text_cleaned,type_
23356,miyako kipas angin kad 927 b,alat-pendingin-ruangan
74914,kacamata 3d vr shinecon virtual reality shinecon,virtual-reality
31351,silent blender ly8001jx heavy duty blender smo...,elektronik-dapur
92386,foomee va03 va02 mouse wired gaming original ...,aksesoris-pc-gaming
39653,lampu led strip anti konslet 2835 10cm led st...,lampu


In [88]:
df_test[["predict_score", "predict_label"]] = df_test["text_cleaned"].apply(lambda x: pd.Series(process.extract(x, keywords, limit=1)[0][1:]))
df_test["predict_label"] = df_test["predict_label"].apply(lambda x: df_group.type_[x])
df_test.head()
# takes about 109 minutes

,text_cleaned,type_,predict_score,predict_label
23356,miyako kipas angin kad 927 b,alat-pendingin-ruangan,86,alat-pendingin-ruangan
74914,kacamata 3d vr shinecon virtual reality shinecon,virtual-reality,86,virtual-reality
31351,silent blender ly8001jx heavy duty blender smo...,elektronik-dapur,86,elektronik-dapur
92386,foomee va03 va02 mouse wired gaming original ...,aksesoris-pc-gaming,36,aksesoris-pc-gaming
39653,lampu led strip anti konslet 2835 10cm led st...,lampu,41,virtual-reality


In [90]:
from sklearn.metrics import classification_report

print(classification_report(df_test['predict_label'], df_test['type_']))

                           precision    recall  f1-score   support

   aksesoris-game-console       0.81      0.30      0.44      3781
      aksesoris-handphone       0.61      0.50      0.55      2864
aksesoris-komputer-laptop       0.26      0.28      0.27      1374
  aksesoris-mobile-gaming       0.58      0.45      0.51      2044
      aksesoris-pc-gaming       0.31      0.35      0.33      1373
         aksesoris-tablet       0.24      0.31      0.27       968
   alat-pendingin-ruangan       0.56      0.44      0.49      1779
                    audio       0.24      0.47      0.31       728
                  cd-game       0.10      0.25      0.15       627
          desktop-mini-pc       0.57      0.40      0.47      2056
         elektronik-dapur       0.58      0.45      0.51      1714
  elektronik-rumah-tangga       0.51      0.43      0.47      1766
             game-console       0.16      0.37      0.22       598
                handphone       0.62      0.57      0.60     

### Another option using cos similiarity + fuzzy wuzzy

In [95]:
from thefuzz import fuzz
from thefuzz import process
import numpy as np

source_title = "iphone 4"
source_price = 4000000
target_title = "iphone 11"
target_price = 10000000
title_similarity = fuzz.partial_token_sort_ratio(source_title, target_title) / 100
price_range = 1 / np.cosh((target_price / source_price) - 1) 
title_similarity, price_range, (0.2 * title_similarity + 0.8 * price_range)

(0.88, 0.4250960349422805, 0.5160768279538245)